In [1]:
def warn(*args, **kwargs):
    pass

import warnings
warnings.warn = warn

from sklearn.feature_extraction.text import CountVectorizer as CV
from sklearn.svm import LinearSVC as SVM
from sklearn.model_selection import train_test_split as TrTeS
from sklearn.metrics import accuracy_score as AS

import re

In [2]:
DATASET_PATH      = "Datasets/"
DATASET_PATH_MORE = "twitter-datasets/"

POS_DATASET  = "train_pos"
NEG_DATASET  = "train_neg"

FULL_DATASETS_EXT = "_full"
EXT = ".txt"

TEST_FILE_ORIGINAL = "test_data.txt"

In [3]:
def simple_clean(t):
    t = t.replace("<user>", "")
    t = t.replace("<url>", "")
    t = t.replace("\n", "")
    t = re.compile("[.;:!\'?,\"()\[\]]").sub("", t.lower())
    
    return t
    
def clean(tweets):
    return [simple_clean(t) for t in tweets]

def read_tweets_file(pos=True, full=True):
    path = DATASET_PATH+DATASET_PATH_MORE
    
    if pos:
        path += POS_DATASET
    else:
        path += NEG_DATASET
    
    if full:
        path += FULL_DATASETS_EXT
    
    path += EXT
    
    tweets = []
    with open(path, 'r') as file:
        for line in file:
            tweets.append(line)
    
    return tweets

def get_tweets(full=True):
    return read_tweets_file(True,full), read_tweets_file(False,full)

def get_submission_tweets():
    tweets = []
    with open(DATASET_PATH+TEST_FILE_ORIGINAL, 'r') as file:
        for line in file:
            tweets.append(",".join(line.split(",")[1:]))
    
    return tweets

def save_submission(filename, predictions):
    with open(DATASET_PATH+filename, 'w') as file:
        file.write("Id,Prediction\n")
        c = 1
        for p in predictions:
            file.write("{},{}\n".format(c,p))
            c += 1

In [4]:
pos, neg = get_tweets()
te       = get_submission_tweets()
pos, neg = clean(pos), clean(neg)
te       = clean(te)
tr       = pos+neg

labels = [1 if i < len(pos) else -1 for i in range(len(pos)+len(neg))]

In [5]:
cv3 = CV(binary=True, ngram_range=(1, 3))
cv3.fit(tr)

tra3 = cv3.transform(tr)
tes3 = cv3.transform(te)

svm3 = SVM(C=0.05, max_iter=1000)
svm3.fit(tra3, labels)

predictions3 = svm3.predict(tes3)

In [6]:
save_submission("svm3.csv", predictions3)

In [7]:
cv4 = CV(binary=True, ngram_range=(1, 4))
cv4.fit(tr)

tra4 = cv4.transform(tr)
tes4 = cv4.transform(te)

svm4 = SVM(C=0.06, max_iter=1000)
svm4.fit(tra4, labels)

predictions4 = svm4.predict(tes4)

In [8]:
save_submission("svm4.csv", predictions4)